In [7]:
#!/usr/bin/env python
# coding: utf-8


# 导入第三方包
import pandas as pd
import numpy as np

import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')



# 读取数据集，具体下载方式可见操作手册
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

sample_submit = pd.read_csv('submit.csv')


# 训练数据及测试数据准备
all_cols = [f for f in train.columns if f not in ['customer_id','loan_default']]

train_x = train[all_cols]
test_x = test[all_cols]

train_y = train['loan_default']

# model = xgb.XGBClassifier(nthread=-1, max_depth=6, n_estimators=30, learning_rate=0.01, colsample_bytree=.9, gamma=1, reg_alpha=4, objective='binary:logistic', eta=0.2,silent=1, subsample=0.8).fit(X_train, Y_train)


# 作为baseline部分仅使用经典的**LightGBM**作为训练模型，我们还能尝试**XGBoost、CatBoost和NN（神经网络）**
clf_name='xgb'
folds = 5
seed = 2021
kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

train = np.zeros(train_x.shape[0])
test = np.zeros(test_x.shape[0])

cv_scores = []
print('************************************ train_x ************************************')
print(np.where(np.isnan(train_x)))
print(np.where(np.isnan(train_x)))

print('************************************ train_y ************************************')
print(np.where(np.isnan(train_y)))
print(np.where(np.isnan(train_y)))
for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
    print('************************************ {} ************************************'.format(str(i+1)))
    trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
    
    # 判断数据
    print(np.where(np.isnan(trn_x)))
    print(np.where(np.isinf(trn_x)))
    
    print(np.where(np.isnan(val_x)))
    print(np.where(np.isinf(val_x)))
    
    clf = xgb.XGBClassifier(nthread=-1, max_depth=6, n_estimators=30, learning_rate=0.01, colsample_bytree=.9, gamma=1, reg_alpha=4, objective='binary:logistic', eta=0.2,silent=1, subsample=0.8)
   
    model = clf.fit(trn_x, trn_y)
    fileObject = open('xgb.pkl', 'wb')
    #保存模型
    pickle.dump(model, fileObject, protocol=4)
    fileObject.close()
    val_pred = model.predict(val_x)
    test_pred = model.predict(test_x)

    # print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])

    train[valid_index] = val_pred
    test += test_pred / kf.n_splits
    cv_scores.append(roc_auc_score(val_y, val_pred))

    print(cv_scores)

print("%s_scotrainre_list:" % clf_name, cv_scores)
print("%s_score_mean:" % clf_name, np.mean(cv_scores))
print("%s_score_std:" % clf_name, np.std(cv_scores))
return train, test






# 预测结果
sample_submit['loan_default'] = lgb_test
sample_submit['loan_default'] = sample_submit['loan_default'].apply(lambda x:1 if x>0.25 else 0).values
sample_submit.to_csv('result_lgb.csv', index=False)

************************************ train_x ************************************
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
************************************ train_y ************************************
(array([], dtype=int64),)
(array([], dtype=int64),)
************************************ 1 ************************************
(array([], dtype=int64), array([], dtype=int64))
(array([12989, 13849, 19294, 21056, 29873, 48098, 51485, 69757, 94322]), array([41, 41, 41, 41, 41, 41, 41, 41, 41]))
(array([], dtype=int64), array([], dtype=int64))
(array([ 4242, 10034, 23146]), array([41, 41, 41]))


XGBoostError: [20:21:10] ../src/data/data.cc:945: Check failed: valid: Input data contains `inf` or `nan`
Stack trace:
  [bt] (0) /home/anaconda3/envs/tp2/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x9133f) [0x7fc0d73fe33f]
  [bt] (1) /home/anaconda3/envs/tp2/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x14e930) [0x7fc0d74bb930]
  [bt] (2) /home/anaconda3/envs/tp2/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x15f8ff) [0x7fc0d74cc8ff]
  [bt] (3) /home/anaconda3/envs/tp2/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x156cf5) [0x7fc0d74c3cf5]
  [bt] (4) /home/anaconda3/envs/tp2/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(XGDMatrixCreateFromMat_omp+0x7c) [0x7fc0d73f604c]
  [bt] (5) /home/anaconda3/envs/tp2/lib/python3.7/lib-dynload/../../libffi.so.7(+0x69ed) [0x7fc185b819ed]
  [bt] (6) /home/anaconda3/envs/tp2/lib/python3.7/lib-dynload/../../libffi.so.7(+0x6077) [0x7fc185b81077]
  [bt] (7) /home/anaconda3/envs/tp2/lib/python3.7/lib-dynl